In [1]:
# import networkx as nx
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
import random
import time

%matplotlib inline

In [2]:
def check_symmetric(a, tol=1e-8):
    return np.allclose(a, a.T, atol=tol)

In [3]:
# 网络数据导入为numpy数组
dataFile_1 = '../data/BA_2000_3.mat'
data_1 = scio.loadmat(dataFile_1)
network_scale_1 = data_1['A'].shape
network_physical = np.array(data_1['A'])
# network_physical = nx.Graph(data['A'])

dataFile_2 = '../data/BA_2000_3_add_400_edges.mat'
data_2 = scio.loadmat(dataFile_2)
network_scale_2  = data_2['B'].shape
network_information = np.array(data_2['B'])
# network_information = nx.Graph(data['A'])

assert network_scale_1 == network_scale_2, "networks\' size don\'t match!"
assert check_symmetric(network_physical), "network_physical doesn\'t symmetry!"
assert check_symmetric(network_information), "network_information doesn\'t symmetry!"
np.sum(np.sum(network_physical, axis=0)==0)

0

In [4]:
# 参数设置

# 公共参数
Monte_Carlo_loop = 20    # 蒙特卡洛模拟次数
time_steps = 50    # 总的时间步数
N = network_scale_1[0]    # 网络节点数
p = random.randint(0, N-1)    # 初始始随机选出一个感染节点，四舍五入

# SIR参数
bata = 0.2    # 感染概率
mu = 0.1    # 免疫概率

Nodes_SIR = np.zeros([Monte_Carlo_loop, time_steps, N])    # N行代表时间步数，T列代表节点数，记录每步所有节点的状态

infective_count = np.zeros([Monte_Carlo_loop, time_steps])    # 每步感染节点数,用于最终统计
infective_count_average = np.zeros([1, time_steps])

immune_count = np.zeros([Monte_Carlo_loop, time_steps])    # 每步免疫节点数,用于最终统计
immune_count_average = np.zeros([1, time_steps])

s_count = np.zeros([Monte_Carlo_loop, time_steps])    # 每步易感节点数,用于最终统计
s_count_average = np.zeros([1, time_steps])

# UAU参数
lamda = 0.4    # 传播率
delta = 0.15    # 遗忘率

Nodes_UAU = np.zeros([Monte_Carlo_loop, time_steps, N])    # 存储网络中每个节点的状态快照
awareness_count = np.zeros([Monte_Carlo_loop, time_steps])    # 每步知晓节点数量,用于最终统计
awareness_count_average = np.zeros([1, time_steps])

# 其他参数
aplha = 0.6    # 信息上传率
sigma_S = 0.7    # S节点在知道信息后的防御系数/感染率衰减, 相当于减少与周围人的接触

In [ ]:
time_start = time.time()

for loop in range(Monte_Carlo_loop):
    
    Nodes_SIR[loop, 0, p] = 1    # 第一步标出初始感染节点的位置
    Nodes_UAU[loop, 0, p] = 1    # 随机初始化一个节点使其处于感染状态，当前可感染状态节点的快照

    for t in range(time_steps-1):

        # UAU演化

        # 找到可以传播的活跃节点
        active_node = np.where((Nodes_UAU[loop, t, :] == 1))[0]    # 找到当前有传播力的节点的位置
        
#         awareness_count[loop] = len(active_node)    # 统计这一刻有传播力节点的数量

        # UAU传播过程
        for i in active_node:

            spread_rate_current = lamda    # 这个暂时放这里，看能不能省掉
            forget_rate_current = delta

            # UAU传播过程
            neighbor_total = np.where(network_information[i, :] == 1)[0]    # 找到第i个有传染力节点的邻居        
            neighbor_listener = np.setdiff1d(neighbor_total, active_node)    # 从neighbor_total去除active_node的节点，应是排除已经感染的邻?

            for j in neighbor_listener:
                p1 = np.random.rand(1)
                if p1 <= spread_rate_current:
                    Nodes_UAU[loop, t+1, j] = 1    # 这个节点已知

            # UAU遗忘过程
            p2 = np.random.rand(1)
            if p2 <= forget_rate_current:
                Nodes_UAU[loop, t+1, i] = 0    # 这个节点遗忘
            else:
                Nodes_UAU[loop, t+1, i] = 1


        # SIR演化
        for i in range(N):

            if Nodes_SIR[loop, t, i] == 0:    # 0代表易感(S)，如果上一步节点i健康，就看他的邻居有没有被感染的

                neighbor_total = np.where((network_physical[i, :] == 1))[0]    # 查找其邻居
                infective_nodes = np.where((Nodes_SIR[loop, t, :] == 1))[0]
                neighbor_infective = np.intersect1d(infective_nodes, neighbor_total)    # 取交集，真正可以感染它的邻居
                Num_neighbor_infective = len(neighbor_infective)    # 统计

                infect_rate_current = bata    # 当前临时感染率
                rate_temp = 1    # 用于计算感染率

                # SIR感染率改变规则
                if Nodes_UAU[loop, t, i] == 0:
                    rate_temp = (1 - infect_rate_current) ** Num_neighbor_infective                 

                elif Nodes_UAU[loop, t, i] == 1:
                    infect_rate_current = infect_rate_current * sigma_S    # 当前临时感染率
                    rate_temp = (1 - infect_rate_current) ** Num_neighbor_infective


                # SIR感染过程
                v1 = 1 - rate_temp    # 这是最终计算的感染率？
                x1 = np.random.rand(1)
                if x1 <= v1:
                    Nodes_SIR[loop, t+1, i] = 1

                    # 感染节点信息上传(知晓)
                    x2 = np.random.rand(1)
                    if  x2 <= aplha:
                        Nodes_UAU[loop, t+1, i] = 1

            # SIR康复过程
            elif Nodes_SIR[loop, t, i] == 1:

                immune_rate_current = mu
                x3 = np.random.rand(1) 
                if x3 <= immune_rate_current:
                    Nodes_SIR[loop, t+1, i] = 2
                else:
                    Nodes_SIR[loop, t+1, i] = 1

            elif Nodes_SIR[loop, t, i] == 2:
                Nodes_SIR[loop, t+1, i] = 2
            
            else:
                print("There is an else condition happened.")

    print("loop {} - time has passed: {} ".format(loop, time.time()-time_start))

loop 0 - time has passed: 5.356717348098755 
loop 1 - time has passed: 10.694443225860596 
loop 2 - time has passed: 16.09295964241028 
loop 3 - time has passed: 23.267768621444702 
loop 4 - time has passed: 28.856887817382812 
loop 5 - time has passed: 34.33221673965454 
loop 6 - time has passed: 39.623026847839355 
loop 7 - time has passed: 44.650580167770386 
loop 8 - time has passed: 51.343719244003296 
loop 9 - time has passed: 56.5537850856781 
loop 10 - time has passed: 61.75782585144043 
loop 11 - time has passed: 67.02772974967957 
loop 12 - time has passed: 72.20987105369568 
loop 13 - time has passed: 77.39300870895386 
loop 14 - time has passed: 82.7197618484497 
loop 15 - time has passed: 88.28986263275146 
loop 16 - time has passed: 95.20337319374084 
loop 17 - time has passed: 100.87025475502014 


In [ ]:
datasave = '../results/result.mat'
scio.savemat(datasave, {'NodesSIR':Nodes_SIR, 'NodeUAU':Nodes_UAU})

# print("Nodes_SIS: ", Nodes_SIS)
# print("Nodes_UAU: ", Nodes_UAU)

In [ ]:
# 处理数据
for i in range(Monte_Carlo_loop):
    awareness_count[i,:] = np.sum(Nodes_UAU[i,:,:], axis=1)
#     infective_count[i,:] = np.sum(Nodes_SIR[i,:,:], axis=1)

    for j in range(time_steps):
        infective_count[i,j] = np.sum(Nodes_SIR[i, j, :] == 1)
        immune_count[i,j] = np.sum(Nodes_SIR[i, j, :] == 2)
        s_count[i,j] = np.sum(Nodes_SIR[i, j, :] == 0)

awareness_count_average = np.mean(awareness_count, axis=0)
infective_count_average = np.mean(infective_count, axis=0)
immune_count_average = np.mean(immune_count, axis=0)
s_count_average = np.mean(s_count, axis=0)

assert infective_count_average.shape == (time_steps,), "infective_count_average\' size don\'t match!"
assert immune_count_average.shape == (time_steps,), "immune_count_average\' size don\'t match!"
assert awareness_count_average.shape == (time_steps,), "awareness_count_average\' size don\'t match!"

# print("infective_count_average: ", infective_count_average)
# print("awareness_count_average: ", awareness_count_average)

## 规模参照
# Nodes_SIR = np.zeros([Monte_Carlo_loop, time_steps, N])
# infective_count = np.zeros([Monte_Carlo_loop, time_steps])
# infective_count_average = np.zeros([1, time_steps])

In [ ]:
plt.plot(s_count_average/N, label='SIR-S')
plt.plot(infective_count_average/N, label='SIR-I')
plt.plot(immune_count_average/N, label='SIR-R')
plt.plot(awareness_count_average/N, label='UAU')

In [ ]:
# 数据可视化
t = (1 : 1 : total_steps);

# SIS统计
plot(t,infective_count_average, '-o', 'color', 'y', 'linewidth', 1.2);
hold on;

# UAU统计
plot(t, awareness_count_average, '-o', 'color', 'k', 'linewidth', 1.2);
hold on;

xlabel('steps');ylabel('density of nodes');
legend('SIS_I(t)','UAU_I(t)');
hold off;

In [ ]:
import time

print ("time.time(): %f " %time.time())

time_start=time.time()
time.sleep(1)
time_end=time.time()
print('totally cost',time_end-time_start)

In [ ]:
a = np.array([[1,2],[3,4],[5,6]])
datasave = './results/result.mat'
scio.savemat(datasave, {'A':a})
data_1 = scio.loadmat(datasave)